In [2]:
import random
import csv
import numpy as np
import pandas as pd
import Loader
import tensorflow as tf
import Transformer
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn import model_selection
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
from sklearn import svm
from sklearn.metrics import accuracy_score
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.losses import MeanSquaredError
#use for dark mode
plt.rcParams["font.family"] = "serif"
plt.style.use('dark_background')

In C:\Users\Tim\anaconda3\envs\tensorflow\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\Tim\anaconda3\envs\tensorflow\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\Tim\anaconda3\envs\tensorflow\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In C:\Users\Tim\anaconda3\envs\tensorflow\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\Tim\anaconda3\envs\tenso

In [6]:
def runBasicSVM(Xtrain, Ytrain, valData):
    maxC=10**-.01
    clf=svm.SVC(
        C=maxC,                          # The regularization parameter
        kernel='rbf',                   # The kernel type used 
        degree=4,                       # Degree of polynomial function 
        gamma='scale',                  # The kernel coefficient
        coef0=0.0,                      # If kernel = 'poly'/'sigmoid'
        shrinking=True,                 # To use shrinking heuristic
        probability=False,              # Enable probability estimates
        tol=0.001,                      # Stopping crierion
        cache_size=200,                 # Size of kernel cache
        class_weight=None,              # The weight of each class
        verbose=False,                  # Enable verbose output
        max_iter=- 1,                   # Hard limit on iterations
        decision_function_shape='ovr',  # One-vs-rest or one-vs-one
        break_ties=False,               # How to handle breaking ties
        random_state=None               # Random state of the model
    )
    #Xtrain,_,valData = Transformer.normalizeData(Xtrain, compoundDataTest, compoundDataValidate, newMean=0, newStd=1)
    clf.fit(Xtrain, Ytrain)
    predictions = clf.predict(Xtrain)
    val_pred = clf.predict(valData)
    return [predictions, val_pred]

def get_data():
    compoundsTrain, smilesTrain, labelsTrain, compoundDataTrain, activitiesTrain = Loader.getTrain(defaultValue=0)
    compoundsTest, smilesTest, labelsTest, compoundDataTest, activitiesTest = Loader.getTest(defaultValue=0)
    compoundsValidate, smilesValidate, labelsValidate, compoundDataValidate, activitiesValidate = Loader.getValidate(defaultValue=0)

    labelsMean, trainMean = Transformer.useAverageFD(labelsTrain, compoundDataTrain)
    _, testMean = Transformer.useAverageFD(labelsTest, compoundDataTest)
    _, valMean = Transformer.useAverageFD(labelsValidate, compoundDataValidate)

    labelsMax, trainMax = Transformer.useMaxFD(labelsTrain, compoundDataTrain)
    _, testMax = Transformer.useMaxFD(labelsTest, compoundDataTest)
    _, valMax = Transformer.useMaxFD(labelsValidate, compoundDataValidate)

    #after transformations are done assign data
    dataLabels = labelsMax
    trainData = trainMax
    testData = testMax
    valData = valMax

    trainData, testData, valData = Transformer.normalizeData(trainData, testData, valData, newMean=0, newStd=1)
    
    classTrain = Transformer.toBinaryClassification(activitiesTrain)
    classVal = Transformer.toBinaryClassification(activitiesValidate)
    classTest = Transformer.toBinaryClassification(activitiesTest)
    
    return trainData, testData, valData, classTrain, classTest, classVal, dataLabels


def parameter_rank(top_x, epochs, avg_std_runs):
    
    trainData, testData, valData, classTrain, classTest, classVal, dataLabels = get_data()
    
    X_train, X_test, y_train, y_test = model_selection.train_test_split(trainData,classTrain,
                                       random_state=104, 
                                       test_size=0.20, 
                                       shuffle=True)
    col_idx = np.arange(0,210)
    probs = []
    for i in range(210):
        probs.append(float(50))
    probs = np.array(probs)
    eval_set = []

    for x in range(epochs):
        
        train_data_df = pd.DataFrame(X_train, columns = dataLabels)
        train_temp_df = train_data_df.copy()
        val_data_df = pd.DataFrame(X_test, columns = dataLabels)
        val_temp_df = val_data_df.copy()

        values = probs
        
        arr1 = values / values.min()
        arr1 = arr1 / arr1.sum()
        #print(arr1, arr1.sum())
        #display(values, sum(values))
        test_cols = np.random.choice(col_idx, top_x, replace=False, p = arr1)
        # using set() to perform task
        test_cols_set = set(test_cols)
        col_idx_set = set(col_idx)
        zero_cols = list(col_idx_set - test_cols_set)
        
        train_temp_df.iloc[:, zero_cols] *= 0
        val_temp_df.iloc[:, zero_cols] *= 0
        
        
        Xtrain = train_temp_df.values
        Xval = val_temp_df.values
        accuracy = runBasicSVM(Xtrain,y_train, Xval)
        curr_val_acc = accuracy_score(y_test, accuracy[1])
        
        if(x < avg_std_runs):
            eval_set.append(curr_val_acc)
        else:
            
            if curr_val_acc > np.average(eval_set) + np.std(eval_set):
                np.multiply.at(probs,test_cols,1.1)
            elif curr_val_acc < np.average(eval_set) - np.std(eval_set):
                np.multiply.at(probs,test_cols,.9)
                prev_val_acc = accuracy[1]
            else:
                prev_val_acc = accuracy[1]
                
            eval_set.pop(0)
            eval_set.append(curr_val_acc)
        
        if((((x+1)/epochs)*100)%10 == 0):
            print(((x+1)/epochs)*100, "% complete")
            #print(probs)

    top_idx = np.flip(probs.argsort()[-210:])
    #display(final_probs,final_probs.argsort()[-5:])
    res_probs_list = [probs[i] for i in top_idx]
    res_feats_list = [dataLabels[i] for i in top_idx]
    result = [res_feats_list, res_probs_list]
    #display(results, results.shape)
    rslt_df = pd.DataFrame(data = np.transpose(result), columns=['Feature','Probability Weight'])
    #sorted_rslt_df = rslt_df.sort_values(by='Probability Weight', ascending=False)
    
    train_data_df = pd.DataFrame(trainData, columns = dataLabels)
    train_temp_df = train_data_df.copy()
    val_data_df = pd.DataFrame(valData, columns = dataLabels)
    val_temp_df = val_data_df.copy()
    
    top_50_idx = probs.argsort()[-50:]
    #test_cols = np.random.choice(col_idx, top_50_idx, replace=False, p = arr1)
    # using set() to perform task
    test_cols_set = set(top_50_idx)
    col_idx_set = set(col_idx)
    zero_cols = list(col_idx_set - test_cols_set)
    
    train_temp_df.iloc[:, zero_cols] *= 0
    val_temp_df.iloc[:, zero_cols] *= 0


    Xtrain = train_temp_df.values
    Xval = val_temp_df.values
    accuracy = runBasicSVM(Xtrain,classTrain, Xval)
    in_samp_acc = accuracy_score(classTrain, accuracy[0])
    out_samp_acc = accuracy_score(classVal, accuracy[1])
    
    final_acc = [in_samp_acc, out_samp_acc]
    
    

    return rslt_df, probs, dataLabels, final_acc
#for x in rslt
results_df, final_probs, features, final_acc = parameter_rank(50,300,20)
display(results_df, final_acc)
        

10.0 % complete
20.0 % complete
30.0 % complete
40.0 % complete
50.0 % complete
60.0 % complete
70.0 % complete
80.0 % complete
90.0 % complete
100.0 % complete


,Feature,Probability Weight
0,peoe_vsa12,419.3315715011778
1,fr_aniline,324.69374507667493
2,slogp_vsa12,295.17613188788624
3,peoe_vsa9,260.3719141769857
4,fr_nh1,246.4113297336056
...,...,...
205,fr_term_acetylene,23.438939584500012
206,bcut2d_mrlow,23.204550188655
207,smr_vsa8,22.51535184350178
208,fr_arn,20.884095169789507


[0.9503128450496872, 0.9205882352941176]

In [7]:
# with pd.option_context('display.max_rows', None,'display.max_columns', None,
#     'display.precision', 3,):
#     display(results)
results_df.head(50)

,Feature,Probability Weight
0,peoe_vsa12,419.3315715011778
1,fr_aniline,324.69374507667493
2,slogp_vsa12,295.17613188788624
3,peoe_vsa9,260.3719141769857
4,fr_nh1,246.4113297336056
5,fpdensitymorgan1,209.87935373793712
6,balabanj,209.87935373793707
7,estate_vsa9,205.70275459855202
8,smr_vsa4,185.13247913869688
9,smr_vsa9,164.953038912579


In [43]:
print(final_probs)
a = [[1,2],["tim","garrett"]]
print(np.transpose(a))

[ 56  27  34  27  42  69  67  33  48  54  24  41  35  64  26  66  27 121
 115  36  36  56  48  73  47  26  49  64 170  27  59  21  40  54  42  95
  21  45  40  28  40 110  27  26  26  43  34  40  49  38 405  83  43  72
 250  38  54  60  53  34  53  36  49  33  40  39  56  34  48  30  90  58
  46 104  46  97  54  25  40 350  40  69  29  22  38  50  43  33  64  24
  64 108  38  32  27  40  60 102  53  49  56  55  58  35  91 133  44  36
  37  33 112  42  41  72  34  51  81  20  35 118  41  18  31  24  16 117
  58  32  19  16  37 111  56 117  57  38  37  33  37  88  88  63  37  53
  74 151 123  38  52  89  52  39  77  50 238  50  22  23  29  36 148 105
  33  31  45  17  33  27  41  47  43  29  20  26  36  72  22  52  37  31
  41  83  35  34 136  54  29  32  35  33  32  56  26  37  24  29  24  70
 474  26  33 116  25  41  38  19  38  33  18 170]
[['1' 'tim']
 ['2' 'garrett']]
